In [3]:
import logging
import os
import time
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from joblib import Parallel, delayed
from scipy.stats import randint, uniform
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import ElasticNet, LinearRegression, Ridge
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit, learning_curve
from sklearn.preprocessing import RobustScaler
from sklearn.svm import SVR
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

# Thiết lập logging và thư mục lưu kết quả
IMG_DIR = 'ml_model_results'
LEARNING_CURVES_DIR = os.path.join(IMG_DIR, 'learning_curves')
os.makedirs(IMG_DIR, exist_ok=True)
os.makedirs(LEARNING_CURVES_DIR, exist_ok=True)

logging.basicConfig(
    filename=os.path.join(IMG_DIR, 'ml_models_log.txt'),
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Cấu hình
CONFIG = {
    'forecast_horizon': 12,
    'seasonal_periods': 12,
    'min_data_length': 24,
    'img_dir': IMG_DIR,
    'results_file': os.path.join(IMG_DIR, 'ml_model_results.csv'),
    'params_file': os.path.join(IMG_DIR, 'best_params.csv'),
    'n_jobs': 1,  # Tắt song song hóa để tránh lỗi
    'lags': list(range(1, 13)),
    'rolling_windows': [3, 6],
    'cv_splits': 5,
    'random_search_iter': 20,  # Tăng số lần lặp để tìm siêu tham số tốt hơn
    'n_features_to_select': 10,
    'correlation_threshold': 0.5,  # Tăng ngưỡng để giữ nhiều đặc trưng hơn
    'nan_threshold': 0.5,
    'pca_variance_ratio': 0.98
}

def create_features(
    data: pd.DataFrame,
    target: str,
    lags: list = CONFIG['lags'],
    rolling_windows: list = CONFIG['rolling_windows'],
    trend_features: bool = False,
    seasonal_features: bool = True,
    fill_method: str = 'interpolate',
    correlation_threshold: float = CONFIG['correlation_threshold'],
    use_pca: bool = True
) -> pd.DataFrame:
    logger.info(f"Tạo đặc trưng cho {target}, kích thước dữ liệu: {data.shape}")

    # Kiểm tra dữ liệu đầu vào
    if len(data) < CONFIG['min_data_length']:
        logger.error(f"Dữ liệu quá ngắn: {len(data)} dòng")
        raise ValueError(f"Dữ liệu quá ngắn: {len(data)} dòng")

    if not isinstance(data.index, pd.DatetimeIndex):
        logger.error("Index của DataFrame phải là DatetimeIndex")
        raise ValueError("Index của DataFrame phải là DatetimeIndex")

    if data.index.duplicated().any():
        logger.error("Index chứa giá trị trùng lặp")
        raise ValueError("Index chứa giá trị trùng lặp")
    if not data.index.is_monotonic_increasing:
        logger.warning("Index không theo thứ tự tăng dần, sắp xếp lại")
        data = data.sort_index()

    exog_vars = ['oil_price', 'gold_price']
    required_cols = [target] + exog_vars
    if not all(col in data.columns for col in required_cols):
        logger.error(f"Thiếu cột: {required_cols}")
        raise ValueError(f"Thiếu cột: {required_cols}")

    df = data.copy()

    # Kiểm tra NaN/Inf
    if df[required_cols].isna().any().any() or np.isinf(df[required_cols]).any().any():
        logger.error(f"Dữ liệu chứa NaN hoặc Inf trong {required_cols}")
        raise ValueError(f"Dữ liệu chứa NaN hoặc Inf trong {required_cols}")

    # Thêm đặc trưng
    for col in required_cols:
        for lag in lags:
            df[f'{col}_lag_{lag}'] = df[col].shift(lag)
        for window in rolling_windows:
            df[f'{col}_roll_mean_{window}'] = df[col].rolling(window=window).mean()

    if trend_features:
        df[f'{target}_diff_1'] = df[target].diff()
        df[f'{target}_trend'] = np.arange(len(df))
        logger.info("Đã thêm đặc trưng xu hướng")

    if seasonal_features:
        df['month'] = df.index.month
        df = pd.get_dummies(df, columns=['month'], prefix='month')
        period = CONFIG['seasonal_periods']
        df['month_sin'] = np.sin(2 * np.pi * df.index.month / period)
        df['month_cos'] = np.cos(2 * np.pi * df.index.month / period)
        logger.info("Đã thêm đặc trưng mùa vụ")

    df['quarter'] = df.index.quarter

    # Xử lý NaN
    for col in df.columns:
        if df[col].isna().all():
            logger.error(f"Cột {col} chỉ chứa NaN")
            raise ValueError(f"Cột {col} chỉ chứa NaN")
        nan_ratio = df[col].isna().mean()
        if nan_ratio > CONFIG['nan_threshold']:
            logger.warning(f"Loại bỏ cột {col} do tỷ lệ NaN cao: {nan_ratio:.2%}")
            df = df.drop(columns=col)
            continue
        if df[col].isnull().any():
            if fill_method == 'interpolate':
                df[col] = df[col].interpolate(method='linear', limit_direction='both')
            elif fill_method == 'ffill':
                df[col] = df[col].ffill()
            elif fill_method == 'bfill':
                df[col] = df[col].bfill()
            if df[col].isnull().any():
                df[col] = df[col].fillna(df[col].mean())
                logger.info(f"Điền NaN trong {col} bằng trung bình: {df[col].mean()}")

    if df.isnull().any().any():
        logger.error(f"Dữ liệu vẫn chứa NaN: {df.isnull().sum().to_dict()}")
        raise ValueError("Dữ liệu vẫn chứa NaN")

    # Loại bỏ đặc trưng tương quan thấp
    if CONFIG.get('check_correlation', True):
        correlations = df.corr()[target].drop(required_cols, errors='ignore')
        low_corr_cols = correlations[abs(correlations) < correlation_threshold].index
        if low_corr_cols.any():
            logger.info(f"Loại bỏ đặc trưng tương quan thấp (< {correlation_threshold}): {list(low_corr_cols)}")
            df = df.drop(columns=low_corr_cols)

    # Áp dụng PCA
    if use_pca and len(df.columns) > len(required_cols):
        feature_cols = [col for col in df.columns if col not in required_cols]
        if feature_cols:
            pca = PCA(n_components=CONFIG['pca_variance_ratio'], svd_solver='full')
            pca.fit(df[feature_cols])
            if sum(pca.explained_variance_ratio_) < 0.6:
                logger.warning("PCA: Tỷ lệ phương sai giải thích thấp, bỏ qua PCA")
            else:
                pca_features = pca.transform(df[feature_cols])
                logger.info(f"PCA: {pca.n_components_} thành phần, tỷ lệ phương sai: {sum(pca.explained_variance_ratio_):.4f}")
                pca_cols = [f'pca_{i+1}' for i in range(pca.n_components_)]
                df_pca = pd.DataFrame(pca_features, index=df.index, columns=pca_cols)
                df = pd.concat([df[required_cols], df_pca], axis=1)
        else:
            logger.warning("Không có đặc trưng để áp dụng PCA")

    logger.info(f"Đặc trưng sau xử lý: {list(df.columns)}")
    return df

def calculate_metrics(actual: np.ndarray, predicted: np.ndarray) -> tuple:
    actual = np.array(actual, dtype=float)
    predicted = np.array(predicted, dtype=float)
    valid_mask = ~np.isnan(actual) & ~np.isnan(predicted) & ~np.isinf(actual) & ~np.isinf(predicted)
    actual, predicted = actual[valid_mask], predicted[valid_mask]

    if len(actual) == 0:
        logger.warning("Không có dữ liệu hợp lệ để tính chỉ số")
        return np.nan, np.nan, np.nan, np.nan, np.nan, np.nan

    rmse = np.sqrt(mean_squared_error(actual, predicted))
    mae = mean_absolute_error(actual, predicted)
    mape = mean_absolute_percentage_error(actual, predicted) * 100 if np.all(np.abs(actual) > 1e-8) else np.nan
    denominator = np.abs(actual) + np.abs(predicted)
    valid_denominator = denominator > 1e-8
    smape = 100 * np.mean(
        2 * np.abs(predicted[valid_denominator] - actual[valid_denominator]) / 
        denominator[valid_denominator]
    ) if valid_denominator.any() else np.nan
    norm_mape = mape / np.mean(np.abs(actual)) if not np.isnan(mape) else np.nan
    directional_acc = np.mean((np.diff(actual) * np.diff(predicted)) > 0) * 100 if len(actual) > 1 else np.nan

    return rmse, mae, mape, smape, norm_mape, directional_acc

def plot_forecast(
    historical: pd.Series,
    test: pd.Series,
    forecast: pd.Series,
    forecast_index: pd.DatetimeIndex,
    title: str,
    ylabel: str,
    filename: str
) -> None:
    plt.figure(figsize=(12, 6))
    plt.plot(historical.index, historical, label='Lịch sử', color='blue')
    plt.plot(test.index, test, label='Thực tế (Test)', color='green')
    plt.plot(forecast_index, forecast, label='Dự báo', color='orange', linestyle='--', linewidth=2)
    plt.title(title)
    plt.xlabel('Thời gian')
    plt.ylabel(ylabel)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    try:
        plt.savefig(os.path.join(CONFIG['img_dir'], filename))
        logger.info(f"Đã lưu biểu đồ: {filename}")
    except Exception as e:
        logger.error(f"Lỗi khi lưu biểu đồ {filename}: {str(e)}")
    finally:
        plt.close()

def plot_comparison_forecasts(
    historical: pd.Series,
    test: pd.Series,
    forecasts: dict,
    forecast_index: pd.DatetimeIndex,
    title: str,
    ylabel: str,
    filename: str,
    metrics: dict = None
) -> None:
    plt.figure(figsize=(14, 8))
    plt.plot(historical.index, historical, label='Lịch sử', color='blue')
    plt.plot(test.index, test, label='Thực tế (Test)', color='green')
    colors = sns.color_palette("husl", len(forecasts))
    for (model_name, forecast), color in zip(forecasts.items(), colors):
        rmse = metrics.get(model_name, {}).get('Test RMSE', np.nan) if metrics else np.nan
        if forecast is None or pd.isna(rmse):
            logger.warning(f"Bỏ qua {model_name} do thiếu dự báo hoặc RMSE")
            continue
        plt.plot(forecast_index, forecast, label=f'{model_name} (RMSE: {rmse:.4f})', linestyle='--', color=color)
    plt.title(title)
    plt.xlabel('Thời gian')
    plt.ylabel(ylabel)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    try:
        plt.savefig(os.path.join(CONFIG['img_dir'], filename))
        logger.info(f"Đã lưu biểu đồ so sánh: {filename}")
    except Exception as e:
        logger.error(f"Lỗi khi lưu biểu đồ so sánh {filename}: {str(e)}")
    finally:
        plt.close()

def plot_learning_curves(
    estimator,
    X: pd.DataFrame,
    y: pd.Series,
    target: str,
    model_name: str,
    filename: str
) -> float:
    try:
        tscv = TimeSeriesSplit(n_splits=CONFIG['cv_splits'])
        train_sizes, train_scores, test_scores = learning_curve(
            estimator, X, y, cv=tscv, scoring='neg_root_mean_squared_error',
            n_jobs=1, train_sizes=np.linspace(0.1, 1.0, 10)
        )
        train_scores_mean = -np.mean(train_scores, axis=1)
        train_scores_std = np.std(train_scores, axis=1)
        test_scores_mean = -np.mean(test_scores, axis=1)
        test_scores_std = np.std(test_scores, axis=1)

        plt.figure(figsize=(10, 6))
        plt.plot(train_sizes, train_scores_mean, label='Train RMSE', color='blue')
        plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                         train_scores_mean + train_scores_std, alpha=0.1, color='blue')
        plt.plot(train_sizes, test_scores_mean, label='Cross-validation RMSE', color='orange')
        plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1, color='orange')
        plt.title(f'Learning Curves - {model_name} ({target})')
        plt.xlabel('Training Examples')
        plt.ylabel('RMSE')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(os.path.join(LEARNING_CURVES_DIR, filename))
        logger.info(f"Đã lưu learning curves: {filename}")
        plt.close()

        overfitting_gap = train_scores_mean[-1] - test_scores_mean[-1]
        logger.info(f"Overfitting gap cho {model_name} ({target}): {overfitting_gap:.4f}")
        return overfitting_gap
    except Exception as e:
        logger.error(f"Lỗi khi vẽ learning curves cho {model_name} ({target}): {str(e)}")
        return np.nan

def select_features(X: np.ndarray, y: np.ndarray, feature_cols: list, k: int = CONFIG['n_features_to_select']) -> list:
    k = min(k, len(feature_cols))
    selector = SelectKBest(score_func=f_regression, k=k)
    try:
        selector.fit(X, y)
        selected = [feature_cols[i] for i in range(len(feature_cols)) if selector.get_support()[i]]
        if not selected:
            logger.warning("Không chọn được đặc trưng, sử dụng tất cả")
            return feature_cols
        logger.info(f"Đặc trưng được chọn: {selected}")
        return selected
    except Exception as e:
        logger.error(f"Lỗi khi chọn đặc trưng: {str(e)}")
        return feature_cols

def optimize_model(
    model,
    param_grid: dict,
    X_train: np.ndarray,
    y_train: np.ndarray,
    n_iter: int = CONFIG['random_search_iter']
) -> tuple:
    tscv = TimeSeriesSplit(n_splits=CONFIG['cv_splits'])
    search = RandomizedSearchCV(
        model, param_grid, n_iter=n_iter, cv=tscv, scoring='neg_root_mean_squared_error',
        n_jobs=1, verbose=1, random_state=42
    )
    try:
        search.fit(X_train, y_train)
        cv_rmse = -search.best_score_
        logger.info(f"Tham số tốt nhất: {search.best_params_}, CV RMSE: {cv_rmse:.4f}")
        return search.best_estimator_, search.best_params_, cv_rmse
    except Exception as e:
        logger.error(f"Lỗi trong optimize_model: {str(e)}")
        return None, {}, np.nan

def run_model(
    train: pd.DataFrame,
    test: pd.DataFrame,
    forecast_index: pd.DatetimeIndex,
    target: str,
    feature_cols: list,
    model_class,
    param_grid: dict = None,
    model_name: str = "Model"
) -> tuple:
    start_time = time.time()
    try:
        X_train = train[feature_cols].dropna()
        y_train = train[target].loc[X_train.index]
        X_test = test[feature_cols].reindex(forecast_index)
        y_test = test[target]

        if len(X_train) < CONFIG['min_data_length']:
            logger.error(f"Tập huấn luyện quá nhỏ: {len(X_train)} mẫu")
            return (None,) * 11

        if not X_test.index.equals(forecast_index):
            logger.error("Index của X_test không khớp với forecast_index")
            return (None,) * 11

        scaler = RobustScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        if np.any(np.isnan(X_train_scaled)) or np.any(np.isinf(X_train_scaled)):
            logger.error(f"X_train_scaled chứa NaN/Inf")
            return (None,) * 11
        if np.any(np.isnan(X_test_scaled)) or np.any(np.isinf(X_test_scaled)):
            logger.error(f"X_test_scaled chứa NaN/Inf")
            return (None,) * 11
        if np.any(np.isnan(y_train)) or np.any(np.isinf(y_train)):
            logger.error(f"y_train chứa NaN/Inf")
            return (None,) * 11

        selected_features = select_features(X_train_scaled, y_train, feature_cols)
        X_train_scaled = pd.DataFrame(X_train_scaled, columns=feature_cols, index=X_train.index)[selected_features]
        X_test_scaled = pd.DataFrame(X_test_scaled, columns=feature_cols, index=X_test.index)[selected_features]

        logger.info(f"Chạy {model_name} cho {target}, X_train shape: {X_train_scaled.shape}")

        model = model_class(random_state=42) if 'random_state' in model_class.__init__.__code__.co_varnames else model_class()
        if param_grid:
            model, best_params, cv_rmse = optimize_model(model, param_grid, X_train_scaled, y_train)
            if model is None:
                logger.error(f"Không thể tối ưu hóa {model_name}")
                return (None,) * 11
        else:
            model.fit(X_train_scaled, y_train)
            tscv = TimeSeriesSplit(n_splits=CONFIG['cv_splits'])
            cv_scores = [
                np.sqrt(mean_squared_error(
                    y_train.iloc[val_idx],
                    model.fit(X_train_scaled.iloc[train_idx], y_train.iloc[train_idx]).predict(X_train_scaled.iloc[val_idx])
                ))
                for train_idx, val_idx in tscv.split(X_train_scaled)
            ]
            cv_rmse = np.mean(cv_scores)
            best_params = {}

        model.fit(X_train_scaled, y_train)
        train_pred = model.predict(X_train_scaled)
        forecast = pd.Series(model.predict(X_test_scaled), index=forecast_index)
        residuals = y_train - train_pred

        train_metrics = calculate_metrics(y_train, train_pred)
        test_metrics = calculate_metrics(y_test, forecast)
        train_rmse, train_mae, train_mape, train_smape, train_norm_mape, train_dir_acc = train_metrics
        test_rmse, test_mae, test_mape, test_smape, test_norm_mape, test_dir_acc = test_metrics

        plot_forecast(
            train[target][-36:], y_test, forecast, forecast_index,
            f'{model_name} Forecast for {target}', target,
            f'{target}_{model_name.lower().replace(" ", "_")}_forecast.png'
        )

        overfitting_gap = plot_learning_curves(
            model, X_train_scaled, y_train, target, model_name,
            f'{target}_{model_name.lower().replace(" ", "_")}_learning_curve.png'
        )

        elapsed_time = time.time() - start_time
        logger.info(
            f"{model_name} ({target}): Train RMSE={train_rmse:.4f}, Test RMSE={test_rmse:.4f}, "
            f"CV RMSE={cv_rmse:.4f}, Test MAE={test_mae:.4f}, Test MAPE={test_mape:.4f}, "
            f"Best Params={best_params}, Overfitting Gap={overfitting_gap:.4f}, Time={elapsed_time:.2f}s"
        )

        return forecast, residuals, train_rmse, test_rmse, cv_rmse, test_mae, test_mape, test_smape, test_norm_mape, test_dir_acc, best_params
    except Exception as e:
        logger.error(f"Lỗi khi chạy {model_name} ({target}): {str(e)}")
        return (None,) * 11

def run_model_for_target(
    target: str,
    train: pd.DataFrame,
    test: pd.DataFrame,
    forecast_index: pd.DatetimeIndex,
    model_name: str,
    model_class,
    feature_cols: list,
    param_grid: dict = None
) -> dict:
    logger.info(f"Chạy {model_name} cho {target}")
    start_time = time.time()
    try:
        result = run_model(train, test, forecast_index, target, feature_cols, model_class, param_grid, model_name)
        forecast, residuals, train_rmse, test_rmse, cv_rmse, test_mae, test_mape, test_smape, test_norm_mape, test_dir_acc, best_params = result

        if forecast is None or test_rmse is None:
            logger.error(f"{model_name} không tạo được dự báo hoặc RMSE")
            return None

        elapsed_time = time.time() - start_time
        logger.info(f"Hoàn thành {model_name} trong {elapsed_time:.2f}s")
        return {
            'Target': target,
            'Model': model_name,
            'Train RMSE': train_rmse,
            'Test RMSE': test_rmse,
            'CV RMSE': cv_rmse,
            'Test MAE': test_mae,
            'Test MAPE': test_mape,
            'Test sMAPE': test_smape,
            'Test NormMAPE': test_norm_mape,
            'Test DirAcc': test_dir_acc,
            'Forecast': forecast,
            'Residuals': residuals,
            'Best Params': best_params
        }
    except Exception as e:
        logger.error(f"Lỗi khi chạy {model_name} ({target}): {str(e)}")
        return None

def main():
    try:
        tf.keras.utils.set_random_seed(42)
        data_path = 'data/data.csv'
        if not os.path.exists(data_path):
            logger.error(f"Tệp dữ liệu {data_path} không tồn tại")
            raise FileNotFoundError(f"Tệp dữ liệu {data_path} không tồn tại")

        data = pd.read_csv(data_path)
        if data.empty:
            logger.error("Tệp dữ liệu rỗng")
            raise ValueError("Tệp dữ liệu rỗng")

        # Kiểm tra dữ liệu đầu vào
        required_columns = ['cpi', 'oil_price', 'gold_price']
        if not all(col in data.columns for col in required_columns):
            logger.error(f"Thiếu cột: {required_columns}")
            raise ValueError(f"Thiếu cột: {required_columns}")

        logger.info(f"Kiểm tra NaN/Inf trong dữ liệu gốc: {data[required_columns].isna().sum().to_dict()}")
        logger.info(f"Kiểm tra Inf trong dữ liệu gốc: {data[required_columns].isin([np.inf, -np.inf]).sum().to_dict()}")

        if len(data) < CONFIG['min_data_length']:
            logger.error(f"Dữ liệu quá ngắn: {len(data)} dòng")
            raise ValueError(f"Dữ liệu quá ngắn: {len(data)} dòng")

        try:
            data['time'] = pd.to_datetime(data['time'])
        except Exception as e:
            logger.error(f"Lỗi khi chuyển đổi cột 'time': {str(e)}")
            raise ValueError(f"Lỗi khi chuyển đổi cột 'time': {str(e)}")

        data.set_index('time', inplace=True)
        if data.index.duplicated().any():
            logger.error("Index chứa giá trị trùng lặp")
            raise ValueError("Index chứa giá trị trùng lặp")
        if not data.index.is_monotonic_increasing:
            logger.warning("Index không theo thứ tự tăng dần, sắp xếp lại")
            data = data.sort_index()

        logger.info(f"Kích thước dữ liệu gốc: {len(data)}")
        features_cpi = create_features(data, 'cpi')

        train_cpi = features_cpi[:-CONFIG['forecast_horizon']]
        test_cpi = features_cpi[-CONFIG['forecast_horizon']:]
        logger.info(f"Kích thước tập dữ liệu: Train={len(train_cpi)}, Test={len(test_cpi)}")

        if len(test_cpi) != CONFIG['forecast_horizon']:
            raise ValueError(f"Kích thước tập kiểm tra không khớp: {CONFIG['forecast_horizon']}")

        forecast_index = pd.date_range(start=test_cpi.index[0], periods=CONFIG['forecast_horizon'], freq='MS')
        feature_cols_cpi = [col for col in train_cpi.columns if col != 'cpi']
        logger.info(f"Đặc trưng cho cpi: {feature_cols_cpi}")

        model_configs = {
            'Random Forest': (RandomForestRegressor, {
                'n_estimators': randint(50, 150),
                'max_depth': [3, 5],
                'min_samples_split': [10, 20],
                'min_samples_leaf': [5, 10],
                'max_features': ['sqrt']
            }),
            'XGBoost': (XGBRegressor, {
                'n_estimators': randint(50, 100),
                'max_depth': [3, 4],
                'min_child_weight': [3, 5],
                'subsample': uniform(0.7, 0.3),
                'colsample_bytree': uniform(0.7, 0.3),
                'reg_lambda': uniform(1.0, 5.0),
                'reg_alpha': uniform(0.5, 1.0),
                'learning_rate': uniform(0.01, 0.05)
            }),
            'LightGBM': (LGBMRegressor, {
                'n_estimators': randint(50, 100),
                'max_depth': [3, 4],
                'min_data_in_leaf': [20, 30],
                'lambda_l1': uniform(0.5, 5.0),
                'lambda_l2': uniform(0.5, 5.0),
                'learning_rate': uniform(0.01, 0.05),
                'subsample': uniform(0.7, 0.3),
                'colsample_bytree': uniform(0.7, 0.3)
            }),
            'SVR': (SVR, {
                'C': uniform(0.1, 50.0),
                'epsilon': uniform(0.01, 0.1),
                'kernel': ['rbf']
            }),
            'ElasticNet': (ElasticNet, {
                'alpha': uniform(0.001, 0.1),
                'l1_ratio': uniform(0.3, 0.7)
            }),
            'Linear Regression': (LinearRegression, None),
            'Ridge': (Ridge, {
                'alpha': uniform(0.5, 50.0)
            })
        }

        results = []
        best_params_list = []
        forecasts_cpi = {}
        metrics_cpi = {}

        logger.info("Chạy các mô hình cho cpi")
        tasks_cpi = [
            delayed(run_model_for_target)(
                'cpi', train_cpi, test_cpi, forecast_index, model_name, model_class, feature_cols_cpi, param_grid
            )
            for model_name, (model_class, param_grid) in model_configs.items()
        ]
        model_results_cpi = Parallel(n_jobs=CONFIG['n_jobs'], verbose=1)(tasks_cpi)

        for result in model_results_cpi:
            if result:
                results.append({
                    'Target': result['Target'],
                    'Model': result['Model'],
                    'Train RMSE': result['Train RMSE'],
                    'Test RMSE': result['Test RMSE'],
                    'CV RMSE': result['CV RMSE'],
                    'Test MAE': result['Test MAE'],
                    'Test MAPE': result['Test MAPE'],
                    'Test sMAPE': result['Test sMAPE'],
                    'Test NormMAPE': result['Test NormMAPE'],
                    'Test DirAcc': result['Test DirAcc']
                })
                forecasts_cpi[result['Model']] = result['Forecast']
                metrics_cpi[result['Model']] = {'Test RMSE': result['Test RMSE']}
                if result['Best Params']:
                    best_params_list.append({
                        'Target': result['Target'],
                        'Model': result['Model'],
                        'Best Params': str(result['Best Params'])
                    })
            else:
                logger.warning(f"Kết quả mô hình cho cpi là None: {result['Model'] if result else 'Unknown'}")

        if forecasts_cpi:
            plot_comparison_forecasts(
                train_cpi['cpi'][-36:], test_cpi['cpi'], forecasts_cpi, forecast_index,
                'Comparison of ML Forecasts for CPI', 'CPI',
                'cpi_ml_model_comparison.png', metrics_cpi
            )
        else:
            logger.warning("Không có dự báo hợp lệ cho cpi")

        results_df = pd.DataFrame(results)
        print(results_df)
        try:
            results_df.to_csv(CONFIG['results_file'], index=False)
            logger.info(f"Kết quả lưu tại {CONFIG['results_file']}")
        except PermissionError as e:
            logger.error(f"Không thể ghi {CONFIG['results_file']}: {str(e)}")
            raise

        best_params_df = pd.DataFrame(best_params_list)
        if not best_params_df.empty:
            try:
                best_params_df.to_csv(CONFIG['params_file'], index=False)
                logger.info(f"Tham số tối ưu lưu tại {CONFIG['params_file']}")
            except PermissionError as e:
                logger.error(f"Không thể ghi {CONFIG['params_file']}: {str(e)}")
                raise

        if forecasts_cpi:
            combined_forecast = pd.DataFrame({'Date': forecast_index})
            for model_name, forecast in forecasts_cpi.items():
                combined_forecast[f'{model_name}_cpi'] = forecast
            try:
                combined_forecast.to_csv(os.path.join(IMG_DIR, 'combined_forecast_cpi.csv'), index=False)
                logger.info(f"Dự báo kết hợp lưu tại {IMG_DIR}/combined_forecast_cpi.csv")
            except PermissionError as e:
                logger.error(f"Không thể ghi combined_forecast_cpi.csv: {str(e)}")
                raise
        else:
            logger.warning("Không có dự báo hợp lệ để lưu")

    except Exception as e:
        logger.error(f"Lỗi chương trình chính: {str(e)}")
        raise

if __name__ == "__main__":
    main()

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Fitting 5 folds for each of 20 candidates, totalling 100 fits
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] lambda_l1 is set=5.25357153204958, reg_alpha=0.0 will be ignored. Current value: lambda_l1=5.25357153204958
[LightGBM] [Warning] lambda_l2 is set=4.159969709057025, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.159969709057025
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] lambda_l1 is set=5.25357153204958, reg_alpha=0.0 will be ignored. Curr

[Parallel(n_jobs=1)]: Done 7 out of 7 | elapsed:   37.3s finished


  Target              Model  Train RMSE  Test RMSE   CV RMSE  Test MAE  \
0    cpi      Random Forest    0.487377   0.301437  0.521091  0.227614   
1    cpi            XGBoost    0.350188   0.228651  0.450697  0.177789   
2    cpi           LightGBM    0.421537   0.199771  0.490759  0.159982   
3    cpi                SVR    0.358082   0.195939  0.465043  0.158854   
4    cpi         ElasticNet    0.467248   0.248994  0.398060  0.201707   
5    cpi  Linear Regression    0.466864   0.249170  0.423919  0.200158   
6    cpi              Ridge    0.466902   0.248871  0.400999  0.199354   

   Test MAPE  Test sMAPE  Test NormMAPE  Test DirAcc  
0   0.226982    0.226865       0.002264    63.636364  
1   0.177172    0.177185       0.001767    81.818182  
2   0.159377    0.159436       0.001590    81.818182  
3   0.158256    0.158277       0.001579    81.818182  
4   0.201139    0.201094       0.002006    81.818182  
5   0.199557    0.199566       0.001991    81.818182  
6   0.198756    0.1987